In [ ]:
%load_ext autoreload
%autoreload 2

#Misc
import time, os, sys, pdb
from glob import glob
from fnmatch import fnmatch

#Base
import numpy as np
import xarray as xr
import pandas as pd
import scipy.stats as st
from multiprocessing import Pool
from sklearn.decomposition import PCA
from tqdm import tqdm

#Save
import json, h5py
import scipy.io as sio

#Plot
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages

#Predictive Coding Github Repository
sys.path.append('/home/dwyrick/Git/openscope_predictive_coding/')
from openscope_predictive_coding.ophys.dataset.openscope_predictive_coding_dataset import OpenScopePredictiveCodingDataset
from openscope_predictive_coding.ophys.response_analysis.response_analysis import ResponseAnalysis

#User
import openscope_predictive_coding.ophys.decoding_analysis.decoding as dc
import openscope_predictive_coding.ophys.response_analysis.utilities as openscope_utils

#Params
mainseq_ids = [68, 78, 13, 26]
oddball_ids = [6, 17, 22, 51, 71, 89, 103, 110, 111, 112]
all_ids = np.concatenate((mainseq_ids,oddball_ids))
SaveDir = '/home/dwyrick/projects/predictive_coding/results/decoding/'
PlotDir = os.path.join(SaveDir,'plots')

areanames = ['VISp', 'VISpm','RSP']
mainseq_labels = ['A','B','C','D']
blocknames = ['randomized_control_pre','oddball','transition_control','randomized_control_post']
stimulus_blocks = ['randomized_control_pre','oddball','transition_control','randomized_control_post']
blocklabels = ['Rand-Ctrl-Pre', 'Oddball', 'Trans-Ctrl', 'Rand-Ctrl-Post']

import warnings
warnings.filterwarnings("ignore")

In [ ]:
color_names=['windows blue',
             'red',
             'amber',
             'faded green',
             'dusty purple',
             'orange',
             'steel blue',
             'pink',
             'greyish',
             'mint',
             'clay',
             'light cyan',
             'forest green',
             'pastel purple',
             'salmon',
             'dark brown',
             'lavender',
             'pale green',
             'dark red',
             'gold',
             'dark teal',
             'rust',
             'fuchsia',
             'pale orange',
             'cobalt blue',
             'mahogany',
             'cloudy blue',
             'dark pastel green',
             'dust',
             'electric lime',
             'fresh green',
             'light eggplant',
             'nasty green']
 
color_palette = sns.xkcd_palette(color_names)
cc = sns.xkcd_palette(color_names)

In [ ]:
# data streams for each imaging session are saved as .h5 files in each folder's experiment in this directory
cache_dir = '/srv/data/AllenInst/opc_analysis'

# Read in experiment manifest
manifest_file = '/srv/data/AllenInst/opc_analysis/opc_production_manifest.xlsx'
manifest = pd.read_excel(manifest_file)

# limit to experiments that passed QC
manifest = manifest[manifest['experiment_state']=='passed']
areanames = np.unique(manifest['imaging_area'])

data = sio.loadmat('/home/dwyrick/projects/predictive_coding/data/preferredStim.mat')
preferred_imageIDs = data['preferred_imageIDs'][0]
cellIDs = data['cellIDs'][0]

cell_metadata_df = pd.read_csv('/home/dwyrick/projects/predictive_coding/data/cell_metadata.csv')
cell_metadata_df.drop(columns='Unnamed: 0',inplace=True)
cell_metadata_df.drop_duplicates(subset='cell_specimen_id',inplace=True)
cell_metadata_df.set_index('cell_specimen_id', drop=True, inplace=True)


pIDs = []
for cellID in cell_metadata_df.index.values:
    
    indy = np.where(cellIDs == cellID)[0][0]
    pID = preferred_imageIDs[indy]
    pIDs.append(pID)
    
cell_metadata_df['pref_stim'] = pIDs

os.chdir('/home/dwyrick/projects/predictive_coding')
os.getcwd()

'/home/dwyrick/projects/predictive_coding'

## Read in Data

In [ ]:
# Concatenate data from all experiments to create pseudopopulation response arrays
# This cell takes a while to run, which is why I've split up the functions of getting the data and selecting the data we want to decode with
variable = 'summed_response'
use_events = True
pseudopop_responses = {b: {} for b in stimulus_blocks}
areanames = ['VISp', 'VISpm','RSP']
for block_str in stimulus_blocks:
    processes = []
    with Pool(20) as p: 
        for area in areanames:
            processes.append(p.apply_async(dc.create_psuedopopulation,args=(manifest,area,block_str,use_events,variable)))

        for area, out in zip(areanames,processes):
            X, stimulus_df = out.get()
            pseudopop_responses[block_str][area] = X 
                        
# for area, sub_list in zip(areanames,processes):
#     for blockstr, result in zip(stimulus_blocks,sub_list):
#         X, stimulus_df = result.get()
#         pseudopop_responses[blockstr][area] = X 
# pool.close()


## Tracing decoding


In [ ]:
randctrl_labeled_rsponses = {'FF': {},'FB': {}}
transctrl_labeled_rsponses = {'FF': {},'FB': {}}
oddball_labeled_rsponses = {'FF': {},'FB': {}}

for ii, area in enumerate(areanames):
    #Loop over depth as well
    for jj,depthstr in enumerate(['superficial','deep']):
        locstr = '{}_{}'.format(area,depthstr)
        cell_ids_perdepth_FF = cell_metadata_df[(cell_metadata_df['location'] == locstr) & (cell_metadata_df['retrogradely_labeled'] == True) & (cell_metadata_df['projection_pathway'] == 'FF')].index.values
        
        if len(cell_ids_perdepth_FF) > 0:
            randctrl_labeled_rsponses['FF'][locstr] = [randctrl_pseudopop_responses[area][0].loc[:,cell_ids_perdepth_FF], randctrl_pseudopop_responses[area][1].loc[:,cell_ids_perdepth_FF]]
            transctrl_labeled_rsponses['FF'][locstr] = transctrl_pseudopop_responses[area].loc[:,cell_ids_perdepth_FF]
            oddball_labeled_rsponses['FF'][locstr] = oddball_pseudopop_responses[area].loc[:,cell_ids_perdepth_FF]
           
        cell_ids_perdepth_FB = cell_metadata_df[(cell_metadata_df['location'] == locstr) & (cell_metadata_df['retrogradely_labeled'] == True) & (cell_metadata_df['projection_pathway'] == 'FB')].index.values
        if len(cell_ids_perdepth_FB) > 0:
            randctrl_labeled_rsponses['FB'][locstr] = [randctrl_pseudopop_responses[area][0].loc[:,cell_ids_perdepth_FB], randctrl_pseudopop_responses[area][1].loc[:,cell_ids_perdepth_FB]]
            transctrl_labeled_rsponses['FB'][locstr] = transctrl_pseudopop_responses[area].loc[:,cell_ids_perdepth_FB]
            oddball_labeled_rsponses['FB'][locstr] = oddball_pseudopop_responses[area].loc[:,cell_ids_perdepth_FB]
            
        

In [ ]:
#Let's decode stimulus blocks from each other based on responses to MS images

method='kfold'
PlotDir = os.path.join(SaveDir,'plots')

nClasses = 4
#Get off diagonal indices
upper_tmp = np.triu_indices(nClasses,k=1)
lower_tmp = np.tril_indices(nClasses,k=-1)
off_diagonal_indy = (np.concatenate((upper_tmp[0],lower_tmp[0])),np.concatenate((upper_tmp[1],lower_tmp[1])))



classifier = 'nearest_neighbor'
SaveDir = '/home/dwyrick/projects/predictive_coding/results/decoding/pseudopopulation'
fstr = 'pseudopop'
PlotDir = os.path.join(SaveDir,'plots')



# fig2, axes2 = plt.subplots(2,3,figsize=(12,8),gridspec_kw={'hspace':0.35},sharey/=True,sharex=True)

for tracing in ['FF','FB']:
    for ii, area in enumerate(areanames):
        cell_ids = oddball_pseudopop_responses[area].coords['cell_specimen_id'].values

        #Loop over depth as well
        for jj,depthstr in enumerate(['superficial','deep']):
            locstr = '{}_{}'.format(area,depthstr)
            if locstr not in oddball_labeled_rsponses[tracing].keys():
                continue
                
            #Get data for this area
            X_randctrl_pre, Y_randctrl_pre, _ = dc.match_trials(randctrl_labeled_rsponses[tracing][locstr][0], stimulus_df, block='randomized_control_pre', trial_type='ABCD')
            X_oddball, Y_oddball, _ = dc.match_trials(oddball_labeled_rsponses[tracing][locstr], stimulus_df, block='oddball', trial_type='ABCD',trial_indy=slice(740,860))
            X_transctrl, Y_transctrl, _ = dc.match_trials(transctrl_labeled_rsponses[tracing][locstr], stimulus_df, block='transition_control', trial_type='ABCD')
            X_randctrl_post, Y_randctrl_post, _ = dc.match_trials(randctrl_labeled_rsponses[tracing][locstr][1], stimulus_df, block='randomized_control_post', trial_type='ABCD')

            confusion_mat = np.zeros((4,nClasses,nClasses))
            confusion_shf = np.zeros((4,nClasses,nClasses))
            confusion_z = np.zeros((4,nClasses,nClasses))

            nNeurons = X_randctrl_pre.shape[-1]
            for iImg, img_ID in enumerate(mainseq_ids):
                #Get stimulus presentation IDs for this particular image from both blocks
                indy_pre = np.where(Y_randctrl_pre == img_ID)[0]
                indy_oddball = np.where(Y_oddball == img_ID)[0]
                indy_transctrl = np.where(Y_transctrl == img_ID)[0]
                indy_post = np.where(Y_randctrl_post == img_ID)[0]

                #Construct X and Y from both blocks
                X = np.vstack((X_randctrl_pre[indy_pre,:],X_oddball[indy_oddball,:],X_transctrl[indy_transctrl,:],X_randctrl_post[indy_post,:]))
                Y = np.hstack([np.repeat(i,30) for i in range(nClasses)])

                confusion_mat[iImg], confusion_shf[iImg], confusion_z[iImg] = dc.cross_validate(X,Y,clabels=[0,1,2,3],classifier=classifier,method=method,plot_shuffle=False,parallel=False)

#     #                 pdb.set_trace()
#             fpath = os.path.join(SaveDir,'StimulusBlocks_MSimgs_{}_{}_{}_{}.mat'.format(locstr,fstr,method,classifier))
#             with h5py.File(fpath,'w') as h5file:
#                 h5file.create_dataset('confusion_mat',data=confusion_mat)
#                 h5file.create_dataset('confusion_shf',data=confusion_shf)
#                 h5file.create_dataset('confusion_z',data=confusion_z)
#                 h5file.create_dataset('area',data=area)
#                 h5file.create_dataset('classifier',data=classifier)
#                 h5file.create_dataset('method',data=method)

            #Plot shuffle distributions
            dc.plot_confusion_matrices(confusion_mat,confusion_z,plot_titles=['A','B','C','D'],class_labels=['rand-pre','OB', 'Trans','rand-post'])
            plt.suptitle('Classifying Stimulus Condition for area {}, {} neurons'.format(locstr, tracing),y=0.95)
#             plt.savefig(os.path.join(PlotDir,'StimulusBlocks_avg-over-ms_{}_{}_{}_{}_{}.png'.format(tracing,fstr,locstr,method,classifier)))

#             #Sparate diagonal and off-diagonal performance
#             diag_hits = np.concatenate([np.diag(confusion_mat[i]) for i in range(len(mainseq_ids))])

#             offdiag_hits = []
#             for iImg in range(len(mainseq_ids)):
#                 cm = confusion_mat[iImg]
#                 cm[np.diag_indices(nClasses)] = np.nan
#                 offdiag_hits.extend([np.nanmean(cm[i,:]) for i in range(nClasses)]);

#             _, pval = st.wilcoxon(diag_hits,offdiag_hits)
#             ax = axes2[jj,ii]
#             ax.set_title(f'{locstr}')
#             ax.boxplot([diag_hits,offdiag_hits])
#             if pval < pval_thresh:
#                 ax.hlines(1,*[1,2],color='k',zorder=0)
#                 ax.vlines(2,*[1,.95],color='k',zorder=0) 
#                 ax.vlines(1,*[1,.95],color='k',zorder=0)
#                 ax.text(1.4,.975,'*',fontdict={'fontweight':'bold','fontsize': 28,'color': 'k'})

#             ax.set_ylim([-0.15,1.15])
#             if jj == 1:
#                 ax.set_xticks([1,2])
#                 ax.set_xticklabels(['Hits','Off-Diag'])
#             if ii == 0:
#                 ax.set_ylabel('Decoding\n Performance')

#     fig2.suptitle('Decoding Between all Stimulus Contexts \nusing responses to main-sequence images',y=0.995)
# #     fig2.savefig(os.path.join(PlotDir,'StimulusBlocks_BOXPLOT_{}_{}_{}.png'.format(fstr,method,classifier)))

In [ ]:
#In the transition control, there are 30 trials per transition type. so, there are 30 trials where DA are presented in series and 30 trials where DX are presented per oddball
#So, given the # of trials per transition type, I created a decoder for each XA type, keeping the DA trials the same
method='L1O'

classifier = 'nearest_neighbor'
SaveDir = '/home/dwyrick/projects/predictive_coding/results/decoding/pseudopopulation'
fstr = 'pseudopop'
PlotDir = os.path.join(SaveDir,'plots')


nClasses = 2
for tracing in ['FF','FB']:
    for area in areanames:
        cell_ids = transctrl_pseudopop_responses[area].coords['cell_specimen_id'].values

        #Loop over depth as well
        for depthstr in ['superficial','deep']:
            locstr = '{}_{}'.format(area,depthstr)
            if locstr not in oddball_labeled_rsponses[tracing].keys():
                continue

            #Get data for this area
            X_oddball, Y_oddball, Y_sort = dc.match_trials(oddball_labeled_rsponses[tracing][locstr], stimulus_df, block='oddball', trial_type='DAXA')
            X_transctrl, Y_transctrl, Y_sort = dc.match_trials(transctrl_labeled_rsponses[tracing][locstr], stimulus_df, block='transition_control', trial_type='DAXA')
            nNeurons = X_transctrl.shape[-1]
            confusion_mat = np.zeros((len(oddball_ids),2,2))
            confusion_shf = np.zeros((len(oddball_ids),2,2))
            confusion_z = np.zeros((len(oddball_ids),2,2))

            for iImg, img_ID in enumerate(oddball_ids):
                #Get stimulus presentation IDs for this particular image from both blocks
                indy_DA = np.where(Y_sort == mainseq_ids[0])[0]
                indy_XA = np.where(Y_sort == img_ID)[0]

                #Construct X and Y from both blocks
                X = np.vstack((X_transctrl[indy_DA,:],X_transctrl[indy_XA,:]))
                Y = np.hstack((np.repeat(0,len(indy_DA)),np.repeat(1,len(indy_XA))))

                confusion_mat[iImg], confusion_shf[iImg], confusion_z[iImg] = dc.cross_validate(X,Y,clabels=[0,1],classifier=classifier,method=method)

    #         fpath = os.path.join(SaveDir,'DAvsXA_TransCtrl-block_{}_{}_{}.mat'.format(locstr,method,classifier))
    #         with h5py.File(fpath,'w') as h5file:
    #             h5file.create_dataset('confusion_mat',data=confusion_mat)
    #             h5file.create_dataset('confusion_shf',data=confusion_shf)
    #             h5file.create_dataset('confusion_z',data=confusion_z)
    #             h5file.create_dataset('area',data=area)
    #             h5file.create_dataset('classifier',data=classifier)
    #             h5file.create_dataset('method',data=method)

            #Plot shuffle distributions
            dc.plot_decoding_accuracy(np.mean(confusion_mat,axis=0),np.mean(confusion_z,axis=0),class_labels=['(D,A)','(X,A)'])

            plt.suptitle('Classifying Transition Type\n Based on image A response; {}, {} {} neurons'.format(locstr,nNeurons,tracing),y=0.925)
#         plt.savefig(os.path.join(PlotDir,'DAvsXA_TransCtrl-block_{}_{}_{}.png'.format(locstr,method,classifier)))

In [ ]:
#In the transition control, there are 30 trials per transition type. so, there are 30 trials where DA are presented in series and 30 trials where DX are presented per oddball
#So, given the # of trials per transition type, I created a decoder for each XA type, keeping the DA trials the same
method='kfold'

classifier = 'SVM'
SaveDir = '/home/dwyrick/projects/predictive_coding/results/decoding/pseudopopulation'
fstr = 'pseudopop'
PlotDir = os.path.join(SaveDir,'plots')


nClasses = 2
for tracing in ['FF','FB']:
    for area in areanames:
        cell_ids = transctrl_pseudopop_responses[area].coords['cell_specimen_id'].values

        #Loop over depth as well
        for depthstr in ['superficial','deep']:
            locstr = '{}_{}'.format(area,depthstr)
            if locstr not in oddball_labeled_rsponses[tracing].keys():
                continue

            #Get data for this area
            X_oddball, Y_oddball, Y_sort = dc.match_trials(oddball_labeled_rsponses[tracing][locstr], stimulus_df, block='oddball', trial_type='DAXA')
#             X_transctrl, Y_transctrl, Y_sort = dc.match_trials(transctrl_labeled_rsponses[tracing][locstr], stimulus_df, block='transition_control', trial_type='DAXA')
            nNeurons = X_oddball.shape[-1]
#             pdb.set_trace()
            confusion_mat, confusion_shf, confusion_z = dc.cross_validate(X_oddball, Y_oddball, Y_sort,nKfold=5,clabels=[0,1],classifier=classifier,method=method)
            
    #         fpath = os.path.join(SaveDir,'DAvsXA_TransCtrl-block_{}_{}_{}.mat'.format(locstr,method,classifier))
    #         with h5py.File(fpath,'w') as h5file:
    #             h5file.create_dataset('confusion_mat',data=confusion_mat)
    #             h5file.create_dataset('confusion_shf',data=confusion_shf)
    #             h5file.create_dataset('confusion_z',data=confusion_z)
    #             h5file.create_dataset('area',data=area)
    #             h5file.create_dataset('classifier',data=classifier)
    #             h5file.create_dataset('method',data=method)

            #Plot shuffle distributions
            dc.plot_decoding_accuracy(confusion_mat,confusion_z,class_labels=['(D,A)','(X,A)'])

            plt.suptitle('Classifying Transition Type\n Based on image A response; {}, {} {} neurons'.format(locstr,nNeurons,tracing),y=0.925)
        plt.savefig(os.path.join(PlotDir,'DAvsXA_Oddball-block_{}_{}_{}_{}.png'.format(tracing,locstr,method,classifier)))

In [ ]:
classifier = 'SVM'
method='kfold'
reduce_dim=False

#Get off diagonal indices
upper_tmp = np.triu_indices(nClasses,k=1)
lower_tmp = np.tril_indices(nClasses,k=-1)
off_diagonal_indy = (np.concatenate((upper_tmp[0],lower_tmp[0])),np.concatenate((upper_tmp[1],lower_tmp[1])))
    
for tracing in ['FF','FB']:
    for area in areanames:
        cell_ids = transctrl_pseudopop_responses[area].coords['cell_specimen_id'].values

        #Loop over depth as well
        for depthstr in ['superficial','deep']:
            locstr = '{}_{}'.format(area,depthstr)
            if locstr not in oddball_labeled_rsponses[tracing].keys():
                continue
                
#             X_randctrl_pre, Y_randctrl_pre, _ = dc.match_trials(randctrl_labeled_rsponses[tracing][locstr][0], stimulus_df, block='randomized_control_pre', trial_type='ABCD')
            X_oddball, Y_oddball, _ = dc.match_trials(oddball_labeled_rsponses[tracing][locstr], stimulus_df, block='oddball', trial_type='ABCD',trial_indy=slice(740,1220))
#             X_transctrl, Y_transctrl, _ = dc.match_trials(transctrl_labeled_rsponses[tracing][locstr], stimulus_df, block='transition_control', trial_type='ABCD')
#             X_randctrl_post, Y_randctrl_post, _ = dc.match_trials(randctrl_labeled_rsponses[tracing][locstr][1], stimulus_df, block='randomized_control_post', trial_type='ABCD')

        
            X = X_oddball; Y = Y_oddball
            nNeurons = X.shape[-1]
            print(np.unique(Y,return_counts=True))
#             pdb.set_trace()
            confusion_mat, confusion_shf, confusion_z = dc.cross_validate(X,Y,clabels=mainseq_ids,nKfold=10,classifier=classifier,method=method)
            
#             fpath = os.path.join(SaveDir,'mainseq_decoding_{}-block_{}_{}_{}.mat'.format(block_str,locstr,method,classifier))
#             with h5py.File(fpath,'w') as h5file:
#                 h5file.create_dataset('confusion_mat',data=confusion_mat)
#                 h5file.create_dataset('confusion_shf',data=confusion_shf)
#                 h5file.create_dataset('confusion_z',data=confusion_z)
#                 h5file.create_dataset('area',data=area)
#                 h5file.create_dataset('classifier',data=classifier)
#                 h5file.create_dataset('method',data=method)

            dc.plot_decoding_accuracy(confusion_mat,confusion_z,class_labels=['A','B','C','D'],clims=[0.1,0.4])

            plt.suptitle('Classifying MS images for {}, {} {} neurons'.format(locstr, nNeurons, tracing),y=0.925,fontsize=14)
            plt.savefig(os.path.join(PlotDir,'mainseq_decoding_oddball-blocks_{}_{}_{}.png'.format(tracing,locstr,classifier)))
        
#              #Sparate diagonal and off-diagonal performance
#             diag_hits = np.diag(confusion_mat)
#             offdiag_hits = confusion_mat[off_diagonal_indy]

#             axes2[jj,ii].set_title(locstr)
#             axes2[jj,ii].boxplot([diag_hits,offdiag_hits])
#             if jj == 1:
#                 axes2[jj,ii].set_xticks([1,2])
#                 axes2[jj,ii].set_xticklabels(['Hits','Off-Diag'])
#             if ii == 0:
#                 axes2[jj,ii].set_ylabel('Decoding Performance')
            
#     fig2.suptitle('Classifying MS images for {}, {} classifer'.format(locstr,classifier),y=0.995)
#     fig2.savefig(os.path.join(PlotDir,'mainseq_decoding_randctrl-blocks_{}_{}.png'.format(method,classifier)))


In [ ]:
classifier = 'nearest_neighbor'

method = 'kfold'
classifier = 'SVM'
fstr = 'pseudopop'
PlotDir = os.path.join(SaveDir,'plots')
nKfold=10
    
for tracing in ['FF','FB']:
    for area in areanames:
        cell_ids = transctrl_pseudopop_responses[area].coords['cell_specimen_id'].values

        #Loop over depth as well
        for depthstr in ['superficial','deep']:
            locstr = '{}_{}'.format(area,depthstr)
            if locstr not in oddball_labeled_rsponses[tracing].keys():
                continue
                
#             X_randctrl_pre, Y_randctrl_pre, _ = dc.match_trials(randctrl_labeled_rsponses[tracing][locstr][0], stimulus_df, block='randomized_control_pre', trial_type='ABCD')
            X_oddball, Y_oddball, Y_sort = dc.match_trials(oddball_labeled_rsponses[tracing][locstr], stimulus_df, block='oddball', trial_type='ABCDX')
#             X_transctrl, Y_transctrl, _ = dc.match_trials(transctrl_labeled_rsponses[tracing][locstr], stimulus_df, block='transition_control', trial_type='ABCD')
#             X_randctrl_post, Y_randctrl_post, _ = dc.match_trials(randctrl_labeled_rsponses[tracing][locstr][1], stimulus_df, block='randomized_control_post', trial_type='ABCD')

            X = X_oddball; Y = Y_oddball
            nNeurons = X.shape[-1]
            print(np.unique(Y,return_counts=True))
#             pdb.set_trace()
            #Use pseudopopulation vectors for decoding
            confusion_mat, confusion_shf, confusion_z = dc.cross_validate(X_oddball,Y_oddball,Y_sort,method=method,nKfold=nKfold,classifier=classifier,clabels=[68, 78, 13, 26, 1])

    #         fpath = os.path.join(SaveDir,'prevMSvsX_decoding_oddball-block_{}_{}.mat'.format(locstr,classifier))
    #         with h5py.File(fpath,'w') as h5file:
    #             h5file.create_dataset('confusion_mat',data=confusion_mat)
    #             h5file.create_dataset('confusion_shf',data=confusion_shf)
    #             h5file.create_dataset('confusion_z',data=confusion_z)
    #             h5file.create_dataset('area',data=area)
    #             h5file.create_dataset('classifier',data=classifier)
    #             h5file.create_dataset('method',data=method)

            dc.plot_decoding_accuracy(confusion_mat,confusion_z,class_labels=['A','B','C','D','X'],title=locstr,annot=True)
            plt.suptitle('ABCD---X\n {} {} nNeurons'.format(nNeurons,tracing))
            plt.savefig(os.path.join(PlotDir,'prevMSvsX_decoding_oddball-blocks_{}-fold_{}-area_{}neurons.png'.format(nKfold,locstr,tracing)))

In [ ]:
classifier = 'nearest_neighbor'

method = 'kfold'
classifier = 'SVM'
fstr = 'pseudopop'
PlotDir = os.path.join(SaveDir,'plots')
nKfold=10
    
for tracing in ['FF','FB']:
    for area in areanames:
        cell_ids = transctrl_pseudopop_responses[area].coords['cell_specimen_id'].values

        #Loop over depth as well
        for depthstr in ['superficial','deep']:
            locstr = '{}_{}'.format(area,depthstr)
            if locstr not in oddball_labeled_rsponses[tracing].keys():
                continue
                
#             X_randctrl_pre, Y_randctrl_pre, _ = dc.match_trials(randctrl_labeled_rsponses[tracing][locstr][0], stimulus_df, block='randomized_control_pre', trial_type='ABCD')
            X_oddball, Y_oddball, Y_sort = dc.match_trials(oddball_labeled_rsponses[tracing][locstr], stimulus_df, block='oddball', trial_type='XABCD')
#             X_transctrl, Y_transctrl, _ = dc.match_trials(transctrl_labeled_rsponses[tracing][locstr], stimulus_df, block='transition_control', trial_type='ABCD')
#             X_randctrl_post, Y_randctrl_post, _ = dc.match_trials(randctrl_labeled_rsponses[tracing][locstr][1], stimulus_df, block='randomized_control_post', trial_type='ABCD')

            X = X_oddball; Y = Y_oddball
            nNeurons = X.shape[-1]
            print(np.unique(Y,return_counts=True))
#             pdb.set_trace()
            #Use pseudopopulation vectors for decoding
            confusion_mat, confusion_shf, confusion_z = dc.cross_validate(X_oddball,Y_oddball,Y_sort,method=method,nKfold=nKfold,classifier=classifier,clabels=[68, 78, 13, 26, 1])

    #         fpath = os.path.join(SaveDir,'prevMSvsX_decoding_oddball-block_{}_{}.mat'.format(locstr,classifier))
    #         with h5py.File(fpath,'w') as h5file:
    #             h5file.create_dataset('confusion_mat',data=confusion_mat)
    #             h5file.create_dataset('confusion_shf',data=confusion_shf)
    #             h5file.create_dataset('confusion_z',data=confusion_z)
    #             h5file.create_dataset('area',data=area)
    #             h5file.create_dataset('classifier',data=classifier)
    #             h5file.create_dataset('method',data=method)

            dc.plot_decoding_accuracy(confusion_mat,confusion_z,class_labels=['A','B','C','D','X'],title=locstr,annot=True)
            plt.suptitle('XABCD\n {} {} nNeurons'.format(nNeurons,tracing))
            plt.savefig(os.path.join(PlotDir,'nextMSvsX_decoding_oddball-blocks_{}-fold_{}-area_{}neurons.png'.format(nKfold,locstr,tracing)))